<a href="https://colab.research.google.com/github/heebo17/Stock-Predicition-using-ML/blob/main/numerai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Numerai Problem

Multi class Classification Problem with 5 Classes

TPU: Tensor Processing Unit is highly-optimised for large batches and CNNs and has the highest training throughput.

GPU: Graphics Processing Unit shows better flexibility and programmability for irregular computations, such as small batches and nonMatMul computations.

# To Do List:


*   Feature Engineering
*   Feature Selecetion
*   Hyperparameter Tuning





## Load data with API from Numerai

In [ ]:
# installing required libraries
# numerapi, for facilitating data download and predictions uploading
# catboost, for modeling and making predictions
!pip install numerapi

In [ ]:
import os
import gc
import csv
import glob
import time
from pathlib import Path

import numerapi

import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
napi = numerapi.NumerAPI(verbosity="info")
# download current dataset
napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

./numerai_dataset_254.zip: 100%|█████████▉| 392M/394M [00:11<00:00, 38.1MB/s]2021-03-08 19:42:06,328 INFO numerapi.base_api: unzipping file...
./numerai_dataset_254.zip: 394MB [00:29, 38.1MB/s]                           

## Helper functions from Numerai Examples

In [ ]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"   #the f" converts the string
PREDICTION_NAME = f"prediction"

BENCHMARK = 0 #???
BAND = 0.2    #???

#-----------------------------------------------------

# Submissions are scored by spearman correlation
def score(df):
    # method="first" breaks ties based on order in array
    return np.corrcoef(
        df[TARGET_NAME],
        df[PREDICTION_NAME].rank(pct=True, method="first")
    )[0, 1]

def correlation(predictions, targets):
    ranked_preds = predictions.rank(pct=True, method="first")
    return np.corrcoef(ranked_preds, targets)[0, 1]

# The payout function
def payout(scores):
    return ((scores - BENCHMARK) / BAND).clip(lower=-1, upper=1)


# Read the csv file into a pandas Dataframe
def read_csv(file_path):
    with open(file_path, 'r') as f:
        column_names = next(csv.reader(f))
        dtypes = {x: np.float16 for x in column_names if
                  x.startswith(('feature', 'target'))}
    return pd.read_csv(file_path, dtype=dtypes)

## Loading and exploring dataset into memory 🖥️

In [ ]:
%%time
print("# Loading data...")
# The training data is used to train your model how to predict the targets.
training_data = read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
# The tournament data is the data that Numerai uses to evaluate your model.
tournament_data = read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")

example_preds = read_csv(os.path.join(latest_round, "example_predictions.csv"))

validation_data = tournament_data[tournament_data.data_type == "validation"]
test_data = tournament_data[tournament_data.data_type == "test"]
live_data = tournament_data[tournament_data.data_type == "live"]

feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

cols = feature_names+[TARGET_NAME]
training_data.head()

# Loading data...
Loaded 310 features
CPU times: user 52.2 s, sys: 3.56 s, total: 55.7 s
Wall time: 55.7 s


In the tournament data, there is 
test data --> era 575
validation data --> 121
live_data --> X

# Feature Engineering

In [ ]:
#Check Stationarity

## Training our model 


In [ ]:
# Seperate train data into train test split
from sklearn.model_selection import train_test_split

X_train = training_data[feature_names].astype(np.float32)
Y_train = training_data[TARGET_NAME].astype(np.float32)


x_train, x_test, y_train, y_test  = train_test_split(X_train,
                                                     Y_train, 
                                                     test_size = 0.999, 
                                                     random_state=17)

print(x_train.shape)

print(y_train.shape)

(501, 310)
(501,)


In [ ]:
# TO DO
# -->corrcoef as scoring
# -->check if objective and eval metric makes sense to be logloss or logistic

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

num_feature=[310,300]
eta = [0.2,0.4,0.6,0.8,1]             #Step size Shrinkage
max_depth= [4,5,6]                    #Max depth of a tree, default 6, increasing will possibly overfit
learning_rate= [0.1, 0.05, 0.15] 
subsample = [1, 0.5]
colsample_bytree = [1]
min_child_weight=[1]


boost = XGBRegressor(objective="reg:logistic",
                     eval_metric="logloss", tree_method= 'gpu_hist',
    
                     validate_parameters=True, nthread=None,)
params = {
    # Parameters that we are going to tune.
    "booster" : ["gbtree", "dart"],  #gblinear uses linear function
    "num_feature": num_feature,
    'max_depth':max_depth,
    'min_child_weight': min_child_weight,
    'eta':eta,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree
}

clf = GridSearchCV(boost, params, 
                     scoring="neg_mean_squared_error",
                    cv=2, n_jobs=-1, verbose=3)
clf.fit(x_train, y_train)
print("best parameters: ",clf.best_params_)
print("best estimator:", clf.best_estimator_)


best_estimator = clf.best_estimator_
#best_estimator.fit(x_train, y_train)

Fitting 2 folds for each of 120 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   24.6s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  3.2min finished


best parameters:  {'booster': 'gbtree', 'colsample_bytree': 1, 'eta': 0.2, 'max_depth': 4, 'min_child_weight': 1, 'num_feature': 310, 'subsample': 0.5}
best estimator: XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.2,
             eval_metric='logloss', gamma=0, importance_type='gain',
             learning_rate=0.1, max_delta_step=0, max_depth=4,
             min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
             nthread=None, num_feature=310, objective='reg:logistic',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.5, tree_method='gpu_hist',
             validate_parameters=True, verbosity=1)


In [ ]:
%%time
MODEL_FILE = "example_model.cbm"

model = best_estimator

if os.path.isfile(MODEL_FILE):
    print("Loading pre-trained model...")
    model.load_model(MODEL_FILE)
else:
    print("Training model...")
    model.fit(x_train, y_train)
    model.save_model(MODEL_FILE)

Training model...
CPU times: user 356 ms, sys: 233 ms, total: 589 ms
Wall time: 587 ms


## Predictions. Evaluation. ➡️

In [ ]:
validation_data.head()

,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,feature_charisma26,...,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
n0003aa52cab36c2,era121,validation,0.25,0.75,0.50,0.50,0.0,0.75,0.5,0.25,0.50,0.5,0.25,0.0,0.25,0.50,0.25,0.00,0.25,1.00,1.00,0.25,1.00,1.00,0.25,0.25,0.0,0.50,0.25,0.75,0.00,0.50,0.25,0.25,0.25,0.50,0.00,0.50,1.00,0.25,...,0.00,0.0,0.25,0.5,0.25,0.25,0.00,0.25,0.00,0.25,0.50,0.50,0.50,0.50,0.00,0.25,0.75,0.25,0.25,0.50,0.25,0.00,0.25,0.50,0.25,0.5,0.25,0.25,1.00,0.75,0.75,0.75,1.00,0.75,0.50,0.5,1.0,0.00,0.00,0.25
n000920ed083903f,era121,validation,0.75,0.50,0.75,1.00,0.5,0.00,0.0,0.75,0.25,0.0,0.75,0.5,0.00,0.25,0.50,0.00,1.00,0.25,0.25,1.00,1.00,0.25,0.75,0.00,0.0,0.75,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.25,0.00,1.00,0.25,...,0.50,0.5,0.25,1.0,0.50,0.25,0.00,0.25,0.50,0.25,1.00,0.25,0.00,0.50,0.75,0.75,0.50,1.00,1.00,0.25,0.50,0.25,0.50,0.50,0.50,0.5,0.25,0.25,0.75,0.50,0.50,0.50,0.75,1.00,0.75,0.5,0.5,0.50,0.50,0.50
n0038e640522c4a6,era121,validation,1.00,0.00,0.00,1.00,1.0,1.00,1.0,1.00,0.50,0.5,1.00,1.0,1.00,0.75,0.50,0.50,1.00,1.00,0.50,0.50,0.00,1.00,0.50,1.00,0.5,1.00,0.50,1.00,0.25,1.00,1.00,1.00,0.50,1.00,1.00,0.75,1.00,1.00,...,0.25,0.5,0.00,0.0,0.00,0.25,0.25,0.00,0.50,0.00,0.00,0.00,0.25,0.00,0.25,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.75,0.0,0.00,0.25,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.0,0.5,0.50,0.00,1.00
n004ac94a87dc54b,era121,validation,0.75,1.00,1.00,0.50,0.0,0.00,0.0,0.50,0.75,1.0,0.75,0.0,0.50,0.00,0.50,0.75,0.50,0.75,0.25,0.75,0.25,0.75,0.25,0.75,1.0,0.50,0.50,0.75,0.50,1.00,0.50,0.25,0.75,0.25,0.75,0.25,0.75,0.75,...,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.25,0.00,0.25,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.25,0.25,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.25,0.00,0.0,0.0,0.25,0.25,0.50
n0052fe97ea0c05f,era121,validation,0.25,0.50,0.50,0.25,1.0,0.50,0.5,0.25,0.25,0.5,0.50,1.0,1.00,1.00,1.00,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.00,0.25,0.0,0.00,0.75,0.25,1.00,0.25,1.00,0.75,0.00,1.00,0.75,0.75,0.75,0.25,...,0.00,0.5,0.50,0.0,0.75,0.50,0.75,0.25,0.25,0.25,0.00,0.25,0.50,0.25,1.00,1.00,1.00,0.00,0.25,0.00,0.00,0.25,0.25,0.75,1.00,1.0,0.75,0.75,0.50,0.50,0.50,0.75,0.00,0.00,0.75,1.0,0.0,0.25,1.00,0.75


In [ ]:
%%time
"""
print("Generating predictions on training data...")
training_preds = model.predict(training_data[feature_names].astype(np.float32))
training_data[PREDICTION_NAME] = training_preds
gc.collect()
"""
print("Generating predictions on tournament data...")
tournament_preds = model.predict(tournament_data[feature_names].astype(np.float32))
tournament_data[PREDICTION_NAME] = tournament_preds

Generating predictions on tournament data...


In [ ]:
# Check the per-era correlations on the training set (in sample)
train_correlations = training_data.groupby("era").apply(score)
print(f"On training the correlation has mean {train_correlations.mean()} and std {train_correlations.std()}")
print(f"On training the average per-era payout is {payout(train_correlations).mean()}")

# Check the per-era correlations on the validation set (out of sample)
validation_data = tournament_data[tournament_data.data_type == "validation"]
validation_correlations = validation_data.groupby("era").apply(score)
print(f"On validation the correlation has mean {validation_correlations.mean()} and "
        f"std {validation_correlations.std()}")
print(f"On validation the average per-era payout is {payout(validation_correlations).mean()}")

In [ ]:
# FEAT_EXPOSURE: How much your model is correlated to the features across era
feature_exposures = validation_data[feature_names].apply(
    lambda d: correlation(validation_data[PREDICTION_NAME], d), axis=0
)
max_per_era = validation_data.groupby("era").apply(
    lambda d: d[feature_names].corrwith(d[PREDICTION_NAME]).abs().max()
)
max_feature_exposure = max_per_era.mean()
print(f"Max Feature Exposure: {max_feature_exposure}")

In [ ]:
tournament_data[PREDICTION_NAME].to_csv(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv")

## Uploading predictions using your API keys 🚀

To create a key for submission only, 

`Settings -> Create API key -> select "Upload Predictions" -> Save`


In [ ]:
# NameOfYourAI
# Add keys between the quotes
public_id = "YourKeys"
secret_key = "YourKeys"
model_id = "YourKeys"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [ ]:
submission_id = napi.upload_predictions(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv", model_id=model_id)

And its done. Congratulations🎉. Your predictions for latest round are submitted! 


Check some information about your latest predictions on [Numerai Tournament]
(https://numer.ai/tournament). It will show some metrics like this,

![Submission](https://cdn-images-1.medium.com/max/600/1*3pb7M7utM21d3RXnhjx5KA.png)

Note: This screenshot is from my other submissions


## Let's check out how well the `example_predictions` perform 💭
You can compare your models with `example_predictions` and try to beat it on some metrics or atlest, you should aim for positive correlation in initial submissions.

In [ ]:
#@title
tournament_data[PREDICTION_NAME]=example_preds['prediction_kazutsugi'].values

In [ ]:
#@title
# Check the per-era correlations on the validation set (out of sample)
validation_data = tournament_data[tournament_data.data_type == "validation"]
validation_correlations = validation_data.groupby("era").apply(score)
print(f"On validation the correlation has mean {validation_correlations.mean()} and "
        f"std {validation_correlations.std()}")
print(f"On validation the average per-era payout is {payout(validation_correlations).mean()}")

## Some useful tips from my experience for using colab efficiently ✨
- You can do simple data exploration without any accelators(GPU/TPU).
- Use GPU/TPU only when everything is ready for execution.
- You can mount your Google Drive to save any work done here.
- Make sure to terminate session if your work is complete and you no longer need that session.


Created by Suraj Parmar

- Numerai: [SurajP](https://numer.ai/surajp)

- Twitter: [@parmarsuraj99](https://twitter.com/parmarsuraj99)


Thanks to [@NJ](https://twitter.com/tasha_jade) and [@MikeP](https://twitter.com/EasyMikeP) for the feedback
